In [2]:
import json
import pathlib
import sys
from shutil import copyfile, move

import keras
import numpy as np

# http://172.20.11.11:8899/tree/news-comments-upvotes
# import util
# import vis
# from config import *
from keras_text.data import Dataset
from keras_text.models import AlexCNN, AttentionRNN, BasicLSTM, StackedRNN, TokenModelFactory, YoonKimCNN
from keras_text.preprocessing import SimpleTokenizer

/home/group7/anaconda3/envs/johenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [1]:
path_embedding='/mnt/data/embeddings/fasttext_guardian_comments/guardian-twokenized-lower-300.vec'
path_data='/mnt/data/group07/johannes/exp_data/politics_1.0'
dir_proc_data='/mnt/data/group07/johannes/ds/proc_data_pol_1.0'

In [5]:
def build_dataset():
    pathlib.Path(dir_proc_data).mkdir(parents=True)

    X_train, y_train = util.load_data(path_data + '/train.csv')

    tokenizer = SimpleTokenizer()

    # onyl build vocab on training data
    tokenizer.build_vocab(X_train)

    util.build_save_data(X_train, y_train, tokenizer,
                         dir_proc_data + '/train.bin', max_len)

    X_val, y_val = util.load_data(path_data + '/val.csv')
    util.build_save_data(X_val, y_val, tokenizer,
                         dir_proc_data + '/val.bin', max_len)

    X_test, y_test = util.load_data(path_data + '/test.csv')
    util.build_save_data(X_test, y_test, tokenizer,
                         dir_proc_data + '/test.bin', max_len)
# build_dataset()

In [8]:
max_len = 400
embedding_dims = 300

epochs = 200
batch_size = 128
lr = 0.001

def build_keras_model():
    optimizer = keras.optimizers.adam(lr=lr)
    word_encoder_model = YoonKimCNN(filter_sizes=[2, 5, 10], num_filters=100, dropout_rate=0)
    # word_encoder_model = AlexCNN(dropout_rate=[0, 0])
    #word_encoder_model = AttentionRNN(encoder_dims=15)
    # word_encoder_model = StackedRNN(hidden_dims=[15, 15])
    # word_encoder_model = BasicLSTM(hidden_dims=20, dropout_rate=0.5)
    
    model = factory.build_model(
        token_encoder_model=word_encoder_model, trainable_embeddings=False)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy', metrics=['accuracy'])
    
def train():
    
    
    # exp_path = util.create_exp_dir(path_data)
    # copyfile('./run.py', exp_path + '/run.py')

    ds_train = Dataset.load(dir_proc_data + '/train.bin')
    X_train, y_train = ds_train.X, ds_train.y

    ds_val = Dataset.load(dir_proc_data + '/val.bin')
    X_val, y_val = ds_val.X, ds_val.y

    # print(ds_train.tokenizer.decode_texts(X_train[:10]))
    # print(y_train[:10])

    # print(ds_train.tokenizer.decode_texts(X_val[:10]))
    # print(y_val[:10])

    factory = TokenModelFactory(
        2, ds_train.tokenizer.token_index, max_tokens=max_len, embedding_path=path_embedding, embedding_dims=embedding_dims)

    


    checkpoint = keras.callbacks.ModelCheckpoint(
        exp_path + "/best.hdf5", monitor='val_acc', verbose=1, save_best_only=True)
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    csv_logger = keras.callbacks.CSVLogger(
        exp_path + '/log.csv', append=True, separator=';')
    callbacks_list = [checkpoint, early_stop, csv_logger]

    model.summary()

    history = model.fit(X_train, y_train, epochs=epochs,
                        batch_size=batch_size, validation_data=(X_val, y_val), callbacks=callbacks_list)

    # vis.plot_history(history, exp_path)

    best_acc = str(max(history.history['val_acc']))[:6]

    # append best acc
    move(exp_path, exp_path + '_' + best_acc)

train()

INFO:keras_text.embeddings:Building embeddings index...
INFO:keras_text.embeddings:Done
INFO:keras_text.embeddings:Loading embeddings for all words in the corpus


NameError: name 'exp_path' is not defined

In [ ]:
def test_data():
    # TODO
    pass